In [1]:
import os.path as osp

In [ ]:
def make_datapath_list(rootpath):
    imgpath_template = osp.join(rootpath, "JPEGImages", "%s.jpg")
    annopath_template = osp.join(rootpath, "Annotations", "%s.xml")

    train_id_names = osp.join(rootpath + "ImageSets/Main/train.txt")
    val_id_names = osp.join(rootpath + "ImageSets/Main/val.txt")